# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
tic_tac_toe = pd.read_csv('tic-tac-toe.csv')
#tic_tac_toe = pd.read_csv('../your-code/tic-tac-toe.csv')
tic_tac_toe


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [3]:
#Convert the categorical values to numeric in all columns:

tic_tac_numeric = pd.get_dummies(tic_tac_toe, columns=["TL", "TM", "TR", "ML", "MM", "MR", "BL", "BM", "BR"],
                               drop_first = True)
tic_tac_numeric['class'] = tic_tac_numeric['class'].replace({True : 1, False : 0})
tic_tac_numeric

,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0
1,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0
2,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1
3,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0
4,1,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1
954,0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,0,1
955,0,1,0,0,1,1,0,0,1,1,0,0,1,0,1,1,0,0,1
956,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,1


In [5]:
tic_tac_inputs = tic_tac_numeric.drop(columns = 'class')
tic_tac_outputs = tic_tac_numeric['class']

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [6]:
# your code here

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

x_train, x_test, y_train, y_test = train_test_split(tic_tac_inputs, tic_tac_outputs)

print(x_train.shape)
print(x_test.shape)


(718, 18)
(240, 18)


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Crear el modelo secuencial
model = Sequential()

# Agregar una capa oculta con activación ReLU
model.add(Dense(16, activation='relu', input_shape=(x_train.shape[1],)))  # Capa de entrada

# Agregar capa de salida con activación Softmax
model.add(Dense(2, activation='softmax'))  # Capa de salida (dos clases)

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo con menos épocas
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluar el modelo con datos de prueba
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Guardar el modelo
model.save('tic-tac-toe.model')

Epoch 1/5
18/18 [==============================] - 1s 14ms/step - loss: 0.7326 - accuracy: 0.5087 - val_loss: 0.7021 - val_accuracy: 0.5486
Epoch 2/5
18/18 [==============================] - 0s 4ms/step - loss: 0.6888 - accuracy: 0.5575 - val_loss: 0.6804 - val_accuracy: 0.5764
Epoch 3/5
18/18 [==============================] - 0s 5ms/step - loss: 0.6630 - accuracy: 0.5819 - val_loss: 0.6648 - val_accuracy: 0.6111
Epoch 4/5
18/18 [==============================] - 0s 4ms/step - loss: 0.6416 - accuracy: 0.6132 - val_loss: 0.6501 - val_accuracy: 0.6181
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 0.7147 - accuracy: 0.5792
Test Loss: 0.7147417664527893
Test Accuracy: 0.5791666507720947


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [19]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('tic-tac-toe.model')

# Generate random indices to select random rows from the test dataset
num_samples = 5
random_indices = np.random.choice(len(y_test), num_samples, replace=False)

# Select random rows from the test dataset
random_x_test = x_test[random_indices]
random_y_test = y_test.iloc[random_indices]  # Use .iloc to select rows by position

# Make predictions
predictions = model.predict(random_x_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Compare predicted labels with true labels
for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"Predicted Label: {predicted_labels[i]}")
    print(f"True Label: {random_y_test.iloc[i]}")
    if predicted_labels[i] == random_y_test.iloc[i]:
        print("Prediction is correct")
    else:
        print("Prediction is incorrect")
    print()

1/1 [==============================] - 0s 101ms/step
Sample 1:
Predicted Label: 0
True Label: 0
Prediction is correct

Sample 2:
Predicted Label: 1
True Label: 1
Prediction is correct

Sample 3:
Predicted Label: 1
True Label: 1
Prediction is correct

Sample 4:
Predicted Label: 1
True Label: 1
Prediction is correct

Sample 5:
Predicted Label: 1
True Label: 1
Prediction is correct



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [20]:
# your code here
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Create the sequential model
model = Sequential()

# Add a hidden layer with ReLU activation and dropout
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.5))  # Adding dropout for regularization

# Add another hidden layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization

# Add output layer with softmax activation
model.add(Dense(2, activation='softmax'))  # Output layer (two classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more epochs
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model with test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Save the model
model.save('tic-tac-toe_improved.model')

Epoch 1/10
9/9 [==============================] - 1s 29ms/step - loss: 0.7514 - accuracy: 0.5557 - val_loss: 0.6196 - val_accuracy: 0.6389
Epoch 2/10
9/9 [==============================] - 0s 9ms/step - loss: 0.7193 - accuracy: 0.6028 - val_loss: 0.6061 - val_accuracy: 0.6389
Epoch 3/10
9/9 [==============================] - 0s 6ms/step - loss: 0.6837 - accuracy: 0.6132 - val_loss: 0.5971 - val_accuracy: 0.6528
Epoch 4/10
9/9 [==============================] - 0s 10ms/step - loss: 0.6570 - accuracy: 0.6202 - val_loss: 0.5909 - val_accuracy: 0.6806
Epoch 5/10
9/9 [==============================] - 0s 7ms/step - loss: 0.6396 - accuracy: 0.6481 - val_loss: 0.5869 - val_accuracy: 0.6875
Epoch 6/10
9/9 [==============================] - 0s 7ms/step - loss: 0.6389 - accuracy: 0.6429 - val_loss: 0.5839 - val_accuracy: 0.7222
Epoch 7/10
9/9 [==============================] - 0s 9ms/step - loss: 0.6487 - accuracy: 0.6411 - val_loss: 0.5840 - val_accuracy: 0.7292
Epoch 8/10
9/9 [================

**Which approach(es) did you find helpful to improve your model performance?**

In [23]:
# your answer here
# Aumento de la complejidad del modelo: Se han añadido más capas y neuronas al modelo. En lugar de una sola capa oculta con 16 neuronas, ahora hay dos capas ocultas con 64 y 32 neuronas respectivamente. Esto permite al modelo aprender patrones más complejos en los datos.

# Regularización mediante Dropout: Se ha agregado la capa Dropout después de cada capa oculta. El Dropout es una técnica de regularización que ayuda a prevenir el sobreajuste al desactivar aleatoriamente una fracción de las unidades de entrada durante el entrenamiento. Esto ayuda a que el modelo generalice mejor a datos nuevos y no vistos.

# Aumento del número de épocas: Se ha aumentado el número de épocas de entrenamiento de 5 a 10. Esto permite que el modelo tenga más oportunidades para ajustarse a los datos de entrenamiento y mejorar su rendimiento.

print('In this improved version, we added two hidden layers with 64 and 32 neurons, respectively, and applied dropout regularization after each hidden layer.Dropout randomly sets a fraction of input units to 0 at each update during training time, which helps prevent overfitting. We also increased the number of epochs to 10 for more training iterations.  Adjust the dropout rate and other hyperparameters as needed based on your data and performance requirements.')

In this improved version, we added two hidden layers with 64 and 32 neurons, respectively, and applied dropout regularization after each hidden layer.Dropout randomly sets a fraction of input units to 0 at each update during training time, which helps prevent overfitting. We also increased the number of epochs to 10 for more training iterations.  Adjust the dropout rate and other hyperparameters as needed based on your data and performance requirements.
